In [ ]:
# Graph lesion deficit mapping model of stroop
# J Ruffle j.ruffle@ucl.ac.uk

In [ ]:
from graph_tool.all import *; import graph_tool.all as gt; import  matplotlib; import math; import numpy; import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib
#matplotlib.use('Agg')
import pandas as pd
import scipy as scipy
import seaborn as sns
from pylab import *
from multiprocessing import  Pool
import subprocess
import scipy.sparse as sparse
from scipy.io import savemat
# sns.set_theme()
import pickle
pd.set_option('display.max_columns', 500)
import glob

import nibabel as nib

from nilearn import plotting
from nilearn import surface
from nilearn import datasets

import os
import shutil

from tqdm.notebook import tqdm 

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

In [ ]:
np.random.seed(0)
gt.seed_rng(0)

In [ ]:
task_options = ['analogical_reasoning','rapm','deductive_reasoning','hayling','proverbs','fluency','stroop']
print(task_options)

In [ ]:
debug=False
print('Debug mode: '+str(debug))

task = task_options[6]
print("Task here will be..."+str(task))

In [ ]:
path = '/media/Data1/NewIQ/jruffle_analysis/mni_masks_444/network_files/'
out = '/media/Data1/NewIQ/jruffle_analysis/results/'
subs = pd.read_csv('/media/Data1/NewIQ/jruffle_analysis/subs.txt',header=None)[0]
print(len(subs))

In [ ]:
df = pd.read_excel('/media/Data1/NewIQ/Analogical_Reasoning_31_03_23.xlsx')
print(df.shape)
print(df.columns)
# df.head()

In [ ]:
df['Anallogical_Reasoning_Task_overall_accuracy_ (% correct)'].describe()

In [ ]:
df2 = pd.read_excel('/media/Data1/NewIQ/Data.xlsx')
print(df2.shape)
print(df2.columns)
# df2.head()

In [ ]:
df2['Analogical_Reasoning_Task_accuracy (% correct)'].describe()

In [ ]:
df3 = pd.read_excel('/media/Data1/NewIQ/RAPM_31_03_23.xlsx')
print(df3.shape)
print(df3.columns)
# df3.head()

In [ ]:
df3['RAPM'].describe()

In [ ]:
df4 = pd.read_excel('/media/Data1/NewIQ/Deductive_reasoning_31_03_23.xlsx')
print(df4.shape)
print(df4.columns)
# df4.head()

In [ ]:
df4['Deductive_Reasoning_Task_overall_accuracy (% correct)'].describe()

In [ ]:
df5 = pd.read_excel('/media/Data1/NewIQ/Additionals.xlsx')
print(df5.shape)
print(df5.columns)
# df5.head()

In [ ]:
hayling = pd.read_excel('/media/Data1/NewIQ/Hayling_3_05_23.xlsx')
# hayling.head()

In [ ]:
hayling['Hayling_overall_scaled_score'].describe()

In [ ]:
proverbs = pd.read_excel('/media/Data1/NewIQ/Proverbs_3_05_23.xlsx')
# proverbs.head()

In [ ]:
proverbs['Proverbs_total_score'].describe()

In [ ]:
fluency = pd.read_excel('/media/Data1/NewIQ/S_Fluency_3_05_23.xlsx')
# fluency.head()

In [ ]:
fluency['S_fluency_correct_words'].describe()

In [ ]:
stroop = pd.read_excel('/media/Data1/NewIQ/Stroop_3_05_23.xlsx')
# stroop.head()

In [ ]:
stroop['Stroop_words_correct'].describe()

In [ ]:
if task=='analogical_reasoning':
    df = df
    target = 'Anallogical_Reasoning_Task_overall_accuracy_ (% correct)'

if task=='rapm':
    df = df3
    target = 'RAPM'
    
if task=='deductive_reasoning':
    df = df4
    target = 'Deductive_Reasoning_Task_overall_accuracy (% correct)'
    
if task=='hayling':
    df = hayling
    target = 'Hayling_overall_scaled_score'
    
if  task=='proverbs':
    df = proverbs
    target = 'Proverbs_total_score'
    
if task=='fluency':
    df = fluency
    target = 'S_fluency_correct_words'
    
if task == 'stroop':
    df = stroop
    target = 'Stroop_words_correct'

In [ ]:
df['Hosp_Number']=''
for i, row in df.iterrows():
    df.loc[i,'Hosp_Number']=row['Original_scan_filename'].split('_')[0].split('.')[0]

out=out+task+'/'
    
isExist = os.path.exists(out)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(out)
   print("The new directory is created!")

In [ ]:
subs = df['Hosp_Number']

print(len(subs))
subs

template_nodes = pd.read_csv(path+'MNI152_T1_nodes.txt',delim_whitespace=True)
template_nodes['coords9dig']=template_nodes['coords9dig'].apply(lambda x: '{0:0>9}'.format(x))
template_nodes

In [ ]:
from nilearn import datasets, image
niimg = nib.load('/media/Data1/NewIQ/jruffle_analysis/mni_masks_444/MNI152_T1_mni444_thresh.nii.gz')

def parallelize_dataframe(df, func, n_cores=40):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def add_features(df):
    df['mni_x'], df['mni_y'], df['mni_z'] = image.coord_transform(df['x'], df['y'], df['z'], niimg.affine)
    bashCommand = 'atlasquery -a "Talairach Daemon Labels" -c '+str(np.array(df['mni_x'])[0])+','+str(np.array(df['mni_y'])[0])+','+str(np.array(df['mni_z'])[0]) ##prepare the subprocess for FSL atlasquery
    region = subprocess.check_output(['bash','-c',bashCommand])
    
    bashCommand = 'atlasquery -a "Harvard-Oxford Cortical Structural Atlas" -c '+str(np.array(df['mni_x'])[0])+','+str(np.array(df['mni_y'])[0])+','+str(np.array(df['mni_z'])[0]) ##prepare the subprocess for FSL atlasquery
    region_harvard_oxford = subprocess.check_output(['bash','-c',bashCommand])
                                                                                                          
    
    df['mni_x']=df['mni_x'].astype(str)
    df['mni_y']=df['mni_y'].astype(str)
    df['mni_z']=df['mni_z'].astype(str)
    
#     df['label0']=region.decode().split('.')[0].rstrip("\n") ##execute it
    df['label1']=region.decode().split('.')[1].rstrip("\n") ##execute it
    df['label2']=region.decode().split('.')[2].rstrip("\n") ##execute it
    df['label3']=region.decode().split('.')[3].rstrip("\n") ##execute it
#     df['label4']=region.decode().split('.')[4].rstrip("\n") ##execute it

    df['harvard_oxford_cortical']=region_harvard_oxford.decode().split(',')[0].rstrip("\n").strip().replace('<b>Harvard-Oxford Cortical Structural Atlas</b><br>','')
    
    return df

template_nodes = parallelize_dataframe(template_nodes, add_features)

template_nodes['MNI coordinates']=1
for i,row in template_nodes.iterrows():
    template_nodes.iloc[i,template_nodes.shape[1]-1]=str(template_nodes.iloc[i,16])+' '+str(template_nodes.iloc[i,17])+' '+str(template_nodes.iloc[i,18])
    
template_nodes['label3_int']=template_nodes['label3']
template_nodes['label3_int'].replace('*',0,inplace=True)
template_nodes['label3_int'].replace('White Matter',1,inplace=True)
template_nodes['label3_int'].replace('Gray Matter',2,inplace=True)
template_nodes['label3_int'].replace('Cerebro-Spinal Fluid',3,inplace=True)

# template_nodes['label3_int'].value_counts()

In [ ]:
g = Graph(directed=False)

v_prop_coords = g.new_vertex_property("string")
v_prop_coords9dig = g.new_vertex_property("string")
v_prop_name_1 = g.new_vertex_property("string")
v_prop_name_2 = g.new_vertex_property("string")
v_prop_name_3 = g.new_vertex_property("string")
v_prop_name_mni = g.new_vertex_property("string")
v_prop_name_mni_x = g.new_vertex_property("string")
v_prop_name_mni_y = g.new_vertex_property("string")
v_prop_name_mni_z = g.new_vertex_property("string")
v_prop_tissue = g.new_vertex_property("int")
v_prop_harvard_label = g.new_vertex_property("string")

for vertexnumber, row in template_nodes.iterrows():
    #node="v{}".format(vertexnumber)
    node = g.add_vertex()
    
    coords = (row['coords'])
    coords9dig=(row['coords9dig'])
    lab1 = (row['label1'])
    lab2=(row['label2'])
    lab3 = (row['label3'])
    mni=(row['MNI coordinates'])
    mni_x=(row['mni_x'])
    mni_y=(row['mni_y'])
    mni_z=(row['mni_z'])
    tissue=(row['label3_int'])
    harvard=(row['harvard_oxford_cortical'])

    v_prop_coords[vertexnumber] = coords
    v_prop_coords9dig[vertexnumber] = coords9dig
    v_prop_name_1[vertexnumber] = lab1
    v_prop_name_2[vertexnumber] = lab2
    v_prop_name_3[vertexnumber] = lab3
    v_prop_name_mni[vertexnumber] = mni
    v_prop_name_mni_x[vertexnumber] = mni_x
    v_prop_name_mni_y[vertexnumber] = mni_y
    v_prop_name_mni_z[vertexnumber] = mni_z
    v_prop_tissue[vertexnumber] = tissue
    v_prop_harvard_label[vertexnumber]=harvard
    

g.vertex_properties["coords"]=v_prop_coords
g.vertex_properties["coords9dig"]=v_prop_coords9dig
g.vertex_properties["lab1"]=v_prop_name_1
g.vertex_properties["lab2"]=v_prop_name_2
g.vertex_properties["lab3"]=v_prop_name_3
g.vertex_properties["mni"]=v_prop_name_mni
g.vertex_properties["mni_x"]=v_prop_name_mni_x
g.vertex_properties["mni_y"]=v_prop_name_mni_y
g.vertex_properties["mni_z"]=v_prop_name_mni_z
g.vertex_properties["tissue"]=v_prop_tissue
g.vertex_properties["label_harvard_oxford"]=v_prop_harvard_label

print(g)

In [ ]:
template_edges = pd.read_csv(path+'MNI152_T1_edges.txt',delim_whitespace=True)
template_edges['CorNode1']=template_edges['CorNode1'].apply(lambda x: '{0:0>9}'.format(x))
template_edges['CorNode2']=template_edges['CorNode2'].apply(lambda x: '{0:0>9}'.format(x))
template_edges

In [ ]:
# DEBUGGING
if debug:
    df = df[df[target]>0].reset_index(drop=True)
    df = df.iloc[:5]
    subs = df['Hosp_Number']
    subs

In [ ]:
nbs_adjacency_target_error = np.zeros(shape=(template_nodes.shape[0],template_nodes.shape[0]))
nbs_adjacency_nart = np.zeros(shape=(template_nodes.shape[0],template_nodes.shape[0]))
nbs_adjacency_weight = np.zeros(shape=(template_nodes.shape[0],template_nodes.shape[0]))

nbs_adjacency_sol = np.zeros(shape=(template_nodes.shape[0],template_nodes.shape[0]))
nbs_adjacency_ischaemia = np.zeros(shape=(template_nodes.shape[0],template_nodes.shape[0]))

counter=0

for j in tqdm(range(len(subs))):

    sub = subs[j]
#     print("Working on ... "+str(sub))
#     print("")
    
    target_error = df.loc[j,target]

    g.set_edge_filter(None)
    g.set_vertex_filter(None)

    node_file = glob.glob(path+'*'+sub+'*nodes.txt')
    edge_file = glob.glob(path+'*'+sub+'*edges.txt')
    
    if len(node_file)>0 and len(edge_file)>0:
        
        sub_nodes = pd.read_csv(node_file[0],delim_whitespace=True)
        sub_nodes['coords9dig']=sub_nodes['coords9dig'].apply(lambda x: '{0:0>9}'.format(x))

        sub_edges = pd.read_csv(edge_file[0],delim_whitespace=True)
        sub_edges['CorNode1']=sub_edges['CorNode1'].apply(lambda x: '{0:0>9}'.format(x))
        sub_edges['CorNode2']=sub_edges['CorNode2'].apply(lambda x: '{0:0>9}'.format(x))


#         print("Commencing edge compilation")

        arr = pd.DataFrame(sub_edges.iloc[:,:3])
        arr = template_edges[template_edges.set_index(['CorNode1','CorNode2']).index.isin(arr.set_index(['CorNode1','CorNode2']).index)]

        target_error_arr = arr.copy()

        #code for binary target
    #     if target_error == 0:
    #         target_error_arr.iloc[:,2]=0
    #     else:
    #         target_error_arr.iloc[:,2]=1

        target_er0ror_arr.iloc[:,2]=target_error_arr.iloc[:,2]/target_error

        adjacency_sub_target_error = sparse.coo_matrix((target_error_arr.iloc[:, 2], (target_error_arr.iloc[:, 3], target_error_arr.iloc[:, 4])), shape=nbs_adjacency_target_error.shape
                                ).todense()


        adjacency_sub_weight = sparse.coo_matrix((arr.iloc[:, 2], (arr.iloc[:, 3], arr.iloc[:, 4])), shape=nbs_adjacency_weight.shape
                                ).todense()


        nbs_adjacency_target_error=nbs_adjacency_target_error+adjacency_sub_target_error

        nbs_adjacency_weight=nbs_adjacency_weight+adjacency_sub_weight

        counter+=1
        
print("")
print("Complete")
print("Number of lesion networks available and utilised: "+str(counter))
print("")
    
print("normalising by lesion covariate")

with np.errstate(divide='ignore', invalid='ignore'):
    nbs_adjacency_target_error[nbs_adjacency_target_error == np.inf] = 0
    nbs_adjacency_target_error = np.nan_to_num(nbs_adjacency_target_error)
    
print("Complete")

In [ ]:
edge_list = np.nonzero(nbs_adjacency_target_error)
print(len(edge_list[0]))

edge_list_df = pd.DataFrame(edge_list[0])
edge_list_df['1']=edge_list[1]

x = np.array(nbs_adjacency_target_error[np.nonzero(nbs_adjacency_target_error)]).flatten()

if not debug:

    # Calculate the percentiles across the x and y dimension
    perc01 = np.percentile(x, 0.5, axis=0, keepdims=True)
    perc99 = np.percentile(x, 99.5, axis=0, keepdims=True)

    # Clip array with different limits across the z dimension
    normalized = np.clip(x, a_min=perc01, a_max=perc99)
    normalized = (normalized-min(normalized))/(max(normalized)-min(normalized))

if debug:
    normalized = x
    
edge_list_df['target_errorweight']=normalized
edge_list_df['lesionweight']=0
edge_list_df['layer']=1
print(edge_list_df.head())
edge_list_df.shape

##threshold to top 10% for debugging...
threshold=75

if not debug:
    thresh=np.percentile(edge_list_df['target_errorweight'], threshold, axis=0, keepdims=True)[0]
    edge_list_df = edge_list_df[edge_list_df['target_errorweight'] > thresh]  
edge_list_df.shape

In [ ]:
edge_list_lw = np.nonzero(nbs_adjacency_weight)
print(len(edge_list_lw[0]))

edge_list_df2 = pd.DataFrame(edge_list_lw[0])
edge_list_df2['1']=edge_list_lw[1]

x = np.array(nbs_adjacency_weight[np.nonzero(nbs_adjacency_weight)]).flatten()

if not debug:

    # Calculate the percentiles across the x and y dimension
    perc01 = np.percentile(x, 0.5, axis=0, keepdims=True)
    perc99 = np.percentile(x, 99.5, axis=0, keepdims=True)

    # Clip array with different limits across the z dimension
    normalized = np.clip(x, a_min=perc01, a_max=perc99)
    normalized = (normalized-min(normalized))/(max(normalized)-min(normalized))
    
if debug:
    normalized = x

edge_list_df2['target_errorweight']=0
edge_list_df2['lesionweight']=normalized
# edge_list_df2['lesionweight']=np.array(nbs_adjacency_weight[np.nonzero(nbs_adjacency_weight)]).flatten()
edge_list_df2['layer']=-1
print(edge_list_df2.head())
edge_list_df2.shape

##threshold to top 10% for debugging...
if not debug:
    thresh=np.percentile(edge_list_df2['lesionweight'], threshold, axis=0, keepdims=True)[0]
    edge_list_df2 = edge_list_df2[edge_list_df2['lesionweight'] > thresh]  
edge_list_df2.shape

In [ ]:
frames = [edge_list_df, edge_list_df2]
edge_list_final = pd.concat(frames)
edge_list_final

In [ ]:
edge_list_final['random_layer']=np.random.randint(len(np.unique(edge_list_final.iloc[:,4])), size=edge_list_final.shape[0])
edge_list_final
sns.histplot(edge_list_final['random_layer'])

In [ ]:
count = np.isinf(edge_list_final).values.sum()
count

In [ ]:
sns.scatterplot(data=edge_list_final,x='layer',y='random_layer')
edge_list_final_corr = edge_list_final.iloc[:,2:].corr()
sns.heatmap(edge_list_final_corr,annot=True)

In [ ]:
edge_list_final = np.array(edge_list_final)
edge_list_final

target_errorweight = g.new_ep("double")
lesionweight = g.new_ep("double")
elayer = g.new_ep("int")
nullelayer = g.new_ep("int")
g.add_edge_list(edge_list_final, eprops=[target_errorweight,lesionweight, elayer,nullelayer])

g.ep["target_error"] = target_errorweight
g.ep["lw"] = lesionweight
g.ep["layer"] = elayer
g.ep["nulllayer"] = nullelayer

In [ ]:
edge_filter = g.new_edge_property("bool") 
for e in g.edges(): 
        if g.ep.layer[e] ==1:
            edge_filter[e] = True 
        else: 
            edge_filter[e] = False 
g.edge_properties["target_errorfilter"]=edge_filter

edge_filter = g.new_edge_property("bool") 
for e in g.edges(): 
        if g.ep.layer[e] <0:
            edge_filter[e] = True 
        else: 
            edge_filter[e] = False 
g.edge_properties["lesionfilter"]=edge_filter

In [ ]:
g_target_error = g.copy()
g_target_error.set_edge_filter(g_target_error.ep.target_errorfilter)
g_target_error.purge_vertices()
g_target_error.purge_edges()
print(g_target_error)
tree = gt.min_spanning_tree(g_target_error,weights=g_target_error.ep.target_error)
g_target_error.ep["minspanningtree"] = tree

weights = pd.DataFrame(np.array(g_target_error.ep.target_error.get_array()))
weights.rename(columns={0:task},inplace=True)

q75, q25 = np.percentile(weights, [75 ,25])

plt.figure(figsize=(2, 5))
# sns.histplot(weights['target_error Weight'],kde=True,log_scale=True,bins=50).set(ylim=(None,None))
sns.histplot(weights[task])
# plt.axvline(q25,c='red',lw=2,linestyle='dashed')
plt.xticks(rotation=30,horizontalalignment="right")
plt.savefig(path+'target_error_edges.png', dpi=600,bbox_inches='tight')

g_target_error.set_edge_filter(g_target_error.ep.minspanningtree)
g_target_error.purge_vertices()
g_target_error.purge_edges()
print(g_target_error)
degree_target_error = g_target_error.degree_property_map("total")

gt.graph_draw(g_target_error,
                vcmap=matplotlib.cm.inferno,edge_gradient=[],
             vertex_fill_color=gt.prop_to_size(degree_target_error),
              vertex_size=gt.prop_to_size(degree_target_error),
              output=out+task+'_error_minspantree.svg')

In [ ]:
weights.value_counts()

In [ ]:
count = np.isinf(weights[task]).values.sum()
print("It contains " + str(count) + " infinite values")

In [ ]:
g_lw = g.copy()
g_lw.set_edge_filter(g_lw.ep.lesionfilter)
g_lw.purge_vertices()
g_lw.purge_edges()
print(g_lw)
tree = gt.min_spanning_tree(g_lw,weights=g_lw.ep.lw)
g_lw.ep["minspanningtree"] = tree

weights = pd.DataFrame(np.array(g_lw.ep.lw.get_array()))
weights.rename(columns={0:'Lesion Weight'},inplace=True)

q75, q25 = np.percentile(weights, [75 ,25])

plt.figure(figsize=(2, 5))
# sns.histplot(weights['target_error Weight'],kde=True,log_scale=True,bins=50).set(ylim=(None,None))
sns.histplot(weights['Lesion Weight'])
# plt.axvline(q25,c='red',lw=2,linestyle='dashed')
plt.xticks(rotation=30,horizontalalignment="right")
plt.savefig(path+'lesion_edges.png', dpi=600,bbox_inches='tight')


g_lw.set_edge_filter(g_lw.ep.minspanningtree)
g_lw.purge_vertices()
g_lw.purge_edges()
print(g_lw)
degree_lw = g_lw.degree_property_map("total")

gt.graph_draw(g_lw,
                vcmap=matplotlib.cm.winter,edge_gradient=[],
             vertex_fill_color=gt.prop_to_size(degree_lw),
              vertex_size=gt.prop_to_size(degree_lw),
              output=out+'lesionweight_minspantree.svg')

In [ ]:
g.set_edge_filter(None)
print(g)
g.ep

In [ ]:
g.save(out+"graph_file_4mm_space_layered_thresh.xml.gz")
print(g)

# HERE

In [ ]:
g =load_graph(out+"graph_file_4mm_space_layered_thresh.xml.gz")
# g.set_edge_filter(None)
# g.set_vertex_filter(None)
print(g)
g.vp

In [ ]:
g.set_vertex_filter(None)
print(g)
deg = g.degree_property_map(deg='total')
g.vp['deg']=deg
deg_filter = g.new_vertex_property("bool") 
for v in g.vertices():
    if deg[v]>0:
        deg_filter[v] = True
    else:
        deg_filter[v] = False
g.vp['deg_filter'] = deg_filter
g.set_vertex_filter(deg_filter)
print(g)
g.purge_vertices()
g.purge_edges()
g.set_vertex_filter(None)
print(g)

v_list = []
for v in g.vertices():
    v_list.append(g.vp['coords9dig'][v])
to_remove = set(list(template_nodes['coords9dig'].values))-set(v_list)
len(to_remove)
template_nodes = template_nodes[template_nodes['coords9dig'].isin(v_list)].reset_index(drop=True)
template_nodes = template_nodes.set_index('coords9dig')
template_nodes = template_nodes.loc[v_list].reset_index()

sns.histplot(g.degree_property_map(deg='total').get_array())

In [ ]:
# LAYERED MODEL

state_norm = gt.minimize_nested_blockmodel_dl(g,state_args=dict(base_type=gt.LayeredBlockState,
                                                         state_args=dict(ec=g.ep.layer,recs = [g.ep.target_error,g.ep.lw],rec_types=["real-normal","discrete-poisson"], layers=True,deg_corr=True)))
state_norm.entropy()

In [ ]:
# LAYERED MODEL

state_exp = gt.minimize_nested_blockmodel_dl(g,state_args=dict(base_type=gt.LayeredBlockState,
                                                         state_args=dict(ec=g.ep.layer,recs = [g.ep.target_error,g.ep.lw],rec_types=["real-exponential","discrete-poisson"], layers=True,deg_corr=True)))
state_exp.entropy()

In [ ]:
# LAYERED MODEL

state_poi = gt.minimize_nested_blockmodel_dl(g,state_args=dict(base_type=gt.LayeredBlockState,
                                                         state_args=dict(ec=g.ep.layer,recs = [g.ep.target_error,g.ep.lw],rec_types=["discrete-poisson","discrete-poisson"], layers=True,deg_corr=True)))
state_poi.entropy()

In [ ]:
if state_norm.entropy() < state_exp.entropy():
    if state_norm.entropy() < state_poi.entropy():
        print("Normal distribution is best, for resampling")
        state=state_norm
        
if state_exp.entropy() < state_norm.entropy():
    if state_exp.entropy() < state_poi.entropy():
        print("Exponential distribution is best, for resampling")
        state=state_exp
        
if state_poi.entropy() < state_norm.entropy():
    if state_poi.entropy() < state_exp.entropy():
        print("Poisson distribution is best, for resampling")
        state=state_poi

In [ ]:
# history = gt.mcmc_anneal(state, beta_range=(1, 10), mcmc_equilibrate_args=dict(force_niter=10),history=True,verbose=True)
# history = gt.mcmc_anneal(state, beta_range=(1, 10),history=True,verbose=True)

if debug:
    iterations=5
if not debug:
    iterations=100

history = gt.mcmc_anneal(state, beta_range=(1, 10), mcmc_equilibrate_args=dict(force_niter=iterations),history=True,verbose=True)

state.entropy()

In [ ]:
plt.figure(figsize=(5, 5))
sns.lineplot(data=history[3],palette=['k'],legend=False)
plt.xlabel('Iteration')
plt.ylabel('Description length (nats)')
plt.savefig(out+'MCMC_entropy.png', dpi=600,bbox_inches='tight')

In [ ]:
state_nullelayer = gt.NestedBlockState(g,base_type=gt.LayeredBlockState,
                                                         state_args=dict(ec=g.ep.nulllayer,recs = [g.ep.target_error,g.ep.lw],rec_types=["real-normal","discrete-poisson"], layers=True,deg_corr=True))

history_null = gt.mcmc_anneal(state_nullelayer, niter=10,beta_range=(1, 10), mcmc_equilibrate_args=dict(force_niter=iterations),history=True,verbose=True)

In [ ]:
print("Non-layer-corrected DL:\t", state_nullelayer.entropy())
print("Layer-corrected DL:\t", state.entropy())
print(u"ln \u039b: ", state.entropy()-state_nullelayer.entropy())

In [ ]:
entropy = pd.DataFrame(np.zeros(shape=(1,2)),columns=['Layered Model','Null Model'])
entropy.iloc[0,1] = state_nullelayer.entropy()
entropy.iloc[0,0] = state.entropy()
entropy=pd.melt(entropy).rename(columns={"variable": "Model", "value": "Entropy"})
entropy.head()

In [ ]:
state.get_levels()

In [ ]:
plt.figure(figsize=(10, 1))
sns.barplot(data=entropy,y='Model',x='Entropy').set(ylabel='')
# plt.xticks([0.0*1e6,0.2*1e6,0.4*1e6,0.6*1e6,0.8*1e6,1.0*1e6,1.2*1e6],rotation=20,horizontalalignment="right")
plt.savefig(out+'model_entropy.png', dpi=600,bbox_inches='tight')

In [ ]:
pickle.dump(state_nullelayer,open(out+'state_nullelayer_thresh.p','wb'))
pickle.dump(state,open(out+'state_layered_thresh.p','wb'))

In [ ]:
g = load_graph(out+"graph_file_4mm_space_layered_thresh.xml.gz")
g

# g.save(path+"graph_file_4mm_space_layered_thresh.xml.gz")
# print(g)

infile = open(out+'state_nullelayer_thresh.p','rb')
state_nullelayer=pickle.load(infile)

infile = open(out+'state_layered_thresh.p','rb')
state=pickle.load(infile)

In [ ]:
def model_equilibration(model):
    print("Beginning MCMC equilibration on "+str(model))
    S1 = model.entropy()
#     model = model.copy(bs=model.get_bs() + [np.zeros(1)] * 2,sampling = True)
    
    def collect_num_groups(s):
        for l, sl in enumerate(s.get_levels()):
            B = sl.get_nonempty_B()
            h[l][B] += 1
        
    h = [np.zeros(g.num_vertices() + 1) for s in model.get_levels()]
    
    a=mcmc_equilibrate(model,multiflip=True,verbose=True,force_niter=iterations,
                    callback=collect_num_groups,history=True,mcmc_args=dict(niter=10))
    
    i=np.array(a).shape[0]
    print('MCMC equilibrated in', i, 'iterations')
    
    S2 = model.entropy()
    print("Improvement:", S2 - S1)
    
    column_names = ["nattempts","nmoves","entropy"]
    df = pd.DataFrame(a,columns=column_names)
    
    return df,h

In [ ]:
state_nullelayer_mcmc=state_nullelayer.copy()
state_nullelayer_mcmc_df,state_nullelayer_mcmc_h=model_equilibration(state_nullelayer_mcmc)

In [ ]:
state_mcmc=state.copy()
state_mcmc_df,state_mcmc_h=model_equilibration(state_mcmc)

In [ ]:
state_nullelayer_mcmc_df['Model']='Null Model'
state_mcmc_df['Model']='Layered Model'

# entropy = state_mcmc_df.merge(state_nullelayer_mcmc_df, how='inner')
entropy=pd.concat([state_mcmc_df, state_nullelayer_mcmc_df], sort=False)
entropy

In [ ]:
g.set_edge_filter(None)
g.set_vertex_filter(None)
state.g.set_edge_filter(None)
state.g.set_vertex_filter(None)

In [ ]:
l0_prop = state.g.new_vertex_property("int")
l0_prop.a = state.project_partition(0,0).get_array()+1
state.g.vertex_properties["l0"]=l0_prop

l1_prop = state.g.new_vertex_property("int")
l1_prop.a = state.project_partition(1,0).get_array()+1
# g.vertex_properties["l1"]=l1_prop
state.g.vertex_properties["l1"]=l1_prop

l2_prop = state.g.new_vertex_property("int")
l2_prop.a = state.project_partition(2,0).get_array()+1
# g.vertex_properties["l2"]=l2_prop
state.g.vertex_properties["l2"]=l2_prop

l3_prop = state.g.new_vertex_property("int")
l3_prop.a = state.project_partition(3,0).get_array()+1
# g.vertex_properties["l3"]=l3_prop
state.g.vertex_properties["l3"]=l3_prop

l4_prop = state.g.new_vertex_property("int")
l4_prop.a = state.project_partition(4,0).get_array()+1
# g.vertex_properties["l4"]=l4_prop
state.g.vertex_properties["l4"]=l4_prop

l5_prop = state.g.new_vertex_property("int")
l5_prop.a = state.project_partition(5,0).get_array()+1
# g.vertex_properties["l5"]=l5_prop
state.g.vertex_properties["l5"]=l5_prop

l6_prop = state.g.new_vertex_property("int")
l6_prop.a = state.project_partition(6,0).get_array()+1
# g.vertex_properties["l6"]=l6_prop
state.g.vertex_properties["l6"]=l6_prop

l7_prop = state.g.new_vertex_property("int")
l7_prop.a = state.project_partition(7,0).get_array()+1
# g.vertex_properties["l7"]=l7_prop
state.g.vertex_properties["l7"]=l7_prop

l8_prop = state.g.new_vertex_property("int")
l8_prop.a = state.project_partition(8,0).get_array()+1
# g.vertex_properties["l8"]=l8_prop
state.g.vertex_properties["l8"]=l8_prop

l9_prop = state.g.new_vertex_property("int")
l9_prop.a = state.project_partition(9,0).get_array()+1
# g.vertex_properties["l9"]=l9_prop
state.g.vertex_properties["l9"]=l9_prop

# g.save(out+"graph_file_4mm_space_layered_thresh.xml.gz")

In [ ]:
ee_eigen,x_eigen = gt.eigenvector(state.g,weight=gt.prop_to_size(state.g.ep.target_error))
# g.vertex_properties["target_error_x_eigen"]=x_eigen
state.g.vp["target_error_x_eigen"]=x_eigen

#betweenness centrality needs to be rescaled as it cannot take negative weights, as done so here...
#vp_between, ep_between = gt.betweenness(g)
vp_between, ep_between = gt.betweenness(state.g,weight=state.g.ep.target_error)
# g.vertex_properties["target_error_vp_between"]=vp_between
state.g.vertex_properties["target_error_vp_between"]=vp_between

#closeness centrality needs to be rescaled as it cannot take negative weights, as done so here...
c = gt.closeness(state.g,weight=state.g.ep.target_error)
# g.vertex_properties["target_error_c"]=c
state.g.vertex_properties["target_error_c"]=c

ee_hits, authority_hits, hub_hits = gt.hits(state.g,weight=state.g.ep.target_error)
# g.vertex_properties["target_error_authority_hits"]=authority_hits
# g.vertex_properties["target_error_hub_hits"]=hub_hits
state.g.vertex_properties["target_error_authority_hits"]=authority_hits
state.g.vertex_properties["target_error_hub_hits"]=hub_hits

In [ ]:
ee_eigen,x_eigen = gt.eigenvector(state.g,weight=gt.prop_to_size(state.g.ep.lw))
# g.vertex_properties["lw_x_eigen"]=x_eigen
state.g.vertex_properties["lw_x_eigen"]=x_eigen

#betweenness centrality needs to be rescaled as it cannot take negative weights, as done so here...
#vp_between, ep_between = gt.betweenness(g)
vp_between, ep_between = gt.betweenness(state.g,weight=state.g.ep.lw)
# g.vertex_properties["lw_vp_between"]=vp_between
state.g.vertex_properties["lw_vp_between"]=vp_between

#closeness centrality needs to be rescaled as it cannot take negative weights, as done so here...
c = gt.closeness(state.g,weight=state.g.ep.lw)
# g.vertex_properties["lw_c"]=c
state.g.vertex_properties["lw_c"]=c

ee_hits, authority_hits, hub_hits = gt.hits(state.g,weight=state.g.ep.lw)
# g.vertex_properties["lw_authority_hits"]=authority_hits
# g.vertex_properties["lw_hub_hits"]=hub_hits
state.g.vertex_properties["lw_authority_hits"]=authority_hits
state.g.vertex_properties["lw_hub_hits"]=hub_hits

In [ ]:
# g.save(out+"graph_file_4mm_space_layered_thresh.xml.gz")

In [ ]:
template_nodes

In [ ]:
centrality = template_nodes.copy()
centrality['target_error_x_eigen']=state.g.vp.target_error_x_eigen.get_array()
centrality['target_error_vp_between']=np.log(state.g.vp.target_error_vp_between.get_array())
centrality['target_error_c']=state.g.vp.target_error_c.get_array()
centrality['target_error_authority_hits']=np.log(state.g.vp.target_error_authority_hits.get_array())
centrality['target_error_hub_hits']=np.log(state.g.vp.target_error_hub_hits.get_array())

centrality['lw_x_eigen']=state.g.vp.lw_x_eigen.get_array()
centrality['lw_vp_between']=np.log(state.g.vp.lw_vp_between.get_array())
centrality['lw_c']=state.g.vp.lw_c.get_array()
centrality['lw_authority_hits']=np.log(state.g.vp.lw_authority_hits.get_array())
centrality['lw_hub_hits']=np.log(state.g.vp.lw_hub_hits.get_array())

centrality['l0']=state.g.vp.l0.get_array()
centrality['l1']=state.g.vp.l1.get_array()
centrality['l2']=state.g.vp.l2.get_array()
centrality['l3']=state.g.vp.l3.get_array()
centrality['l4']=state.g.vp.l4.get_array()
centrality['l5']=state.g.vp.l5.get_array()
centrality['l6']=state.g.vp.l6.get_array()
centrality['l7']=state.g.vp.l7.get_array()
centrality['l8']=state.g.vp.l8.get_array()
centrality['l9']=state.g.vp.l9.get_array()

centrality.reset_index(drop=True,inplace=True)

centrality.replace([-np.inf], np.nan, inplace=True)

centrality['target_error_x_eigen'].fillna(value=centrality['target_error_x_eigen'].min(),inplace=True)
centrality['target_error_vp_between'].fillna(value=centrality['target_error_vp_between'].min(),inplace=True)
centrality['target_error_c'].fillna(value=centrality['target_error_c'].min(),inplace=True)
centrality['target_error_authority_hits'].fillna(value=centrality['target_error_authority_hits'].min(),inplace=True)
centrality['target_error_hub_hits'].fillna(value=centrality['target_error_hub_hits'].min(),inplace=True)

centrality['lw_x_eigen'].fillna(value=centrality['lw_x_eigen'].min(),inplace=True)
centrality['lw_vp_between'].fillna(value=centrality['lw_vp_between'].min(),inplace=True)
centrality['lw_c'].fillna(value=centrality['lw_c'].min(),inplace=True)
centrality['lw_authority_hits'].fillna(value=centrality['lw_authority_hits'].min(),inplace=True)
centrality['lw_hub_hits'].fillna(value=centrality['lw_hub_hits'].min(),inplace=True)

centrality[['target_error_x_eigen','target_error_vp_between','target_error_c','target_error_authority_hits','target_error_hub_hits',
            'lw_x_eigen','lw_vp_between','lw_c','lw_authority_hits','lw_hub_hits']]+=0.1

centrality.to_csv(out+'centrality_metrics.csv')

centrality

In [ ]:
empty_mask = nib.load('/media/Data1/NewIQ/jruffle_analysis/mni_masks_444/empty_444.nii.gz')

In [ ]:
array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,25]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_target_error_eigen.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,26]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_target_error_between.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,27]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_target_error_closeness.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,28]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_target_error_authority.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,29]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_target_error_hub.nii.gz')

array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,30]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_lw_eigen.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,31]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_lw_between.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,32]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_lw_closeness.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,33]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_lw_authority.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,34]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_lw_hub.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,35]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l0.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,36]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l1.nii.gz')

array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,37]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l2.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,38]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l3.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,39]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l4.nii.gz')


array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,40]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l5.nii.gz')

array_data = np.zeros(shape=empty_mask.dataobj.shape)

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,41]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l6.nii.gz')

for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,42]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l7.nii.gz')


for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,43]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l8.nii.gz')


for i, row in centrality.iterrows():
#     print(i)
    x,y,z = (centrality.loc[i,'x']-1),centrality.loc[i,'y'],centrality.loc[i,'z']
    z-=1
    y-=1
    value = centrality.iloc[i,44]
    array_data[x,y,z]=value
    
array_image=nib.Nifti1Image(array_data,empty_mask.affine)
nib.save(array_image,out+'layered_sbm_l9.nii.gz')

In [ ]:
state.g.set_edge_filter(None)

range(9)
levels = [3]

# 1/np.exp(model_comparison_prediction['target_error'])

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

rowsnum = 0

for l in range(9):
    block = state.project_partition(l,0)
    rowsnum += len(np.unique(np.array(block.get_array())))
    
rowsnum

parcels = pd.DataFrame(np.zeros(shape=(rowsnum,9)),columns=['Level','Block label','Target Error Posterior Mean',
                                                            'Target error 95% CI lower','Target error 95% CI upper',
                                                            'Lesion Weight Posterior Mean','Lesion Weight 95% CI lower',
                                                            'Lesion Weight 95% CI upper',"Significant"])
parcels

row=0

for l in range(9):
# for l in levels:
    print("hierarchy")
    print(l)
    
    block = state.project_partition(l,0)
    print("blocks...")
    print(block.get_array())
    print("")
    print("unique blocks...")
    print(len(np.unique(np.array(block.get_array()))))
    print("")

    print(state.g)
    
    template_path = out+'layered_sbm_l'+str(l)+'.nii.gz'
    template = nib.load(template_path)
    template.affine
    posterior = np.asanyarray(template.dataobj)
    sd = np.asanyarray(template.dataobj)
    posterior_lw = posterior.copy()
    sd_lw = sd.copy()
    
    counter=1
    for b in np.unique(np.array(block.get_array())):
        print("working on block..." +str(b)+": "+str(counter)+"/"+str(len(np.unique(np.array(block.get_array()))))) 
        
        block_filter = state.g.new_vertex_property("bool") 
        for v in state.g.vertices(): 
                if block[v] ==b:
                    block_filter[v] = True 
                else: 
                    block_filter[v] = False 

        g_block = state.g.copy()
        g_block.set_vertex_filter(block_filter)
        g_block.purge_vertices()
        g_block.purge_edges()

#         block_weights = g_block.ep.target_error.get_array() / g_block.ep.lw.get_array()
        block_weights = np.array(g_block.ep.target_error.get_array())
        block_weights_m = np.mean(block_weights)
#         print(block_weights_m)
        block_weights_std = numpy.std(block_weights)
        
        block_weights_lw = np.array(g_block.ep.lw.get_array())
        block_weights_lw_m = np.mean(block_weights_lw)
#         print(block_weights_lw_m)
        block_weights_lw_std = numpy.std(block_weights_lw)
    
        b+=1 ###because we had to increment by 1 for the niftis
        posterior[posterior==b] = block_weights_m
        sd[sd==b] = block_weights_std
        
        posterior_lw[posterior_lw==b] = block_weights_lw_m
        sd_lw[sd_lw==b] = block_weights_lw_std        
        
        parcels.iloc[row,0]=l
        parcels.iloc[row,1]=b
        parcels.iloc[row,2:5] = mean_confidence_interval(block_weights)
        parcels.iloc[row,5:8] = mean_confidence_interval(block_weights_lw)
        
        a1, a2 = parcels.iloc[row,3:5].values
        b1, b2 = parcels.iloc[row,6:8].values

        if max(a2, b2) - min(a1, b1) >= (a2 - a1) + (b2 - b1):
            parcels.iloc[row,8]="*"
            
        if max(a2, b2) - min(a1, b1) < (a2 - a1) + (b2 - b1):
            parcels.iloc[row,8]="NSD"
        
        counter+=1
        row+=1
        
    posteriornifti = nib.Nifti1Image(posterior,template.affine)
    nib.save(posteriornifti,out+'layered_sbm_l'+str(l)+'_posterior_mean.nii.gz')

    sdnifti = nib.Nifti1Image(sd,template.affine)
    nib.save(sdnifti,out+'layered_sbm_l'+str(l)+'_posterior_sd.nii.gz')
    
    
    posteriornifti_lw = nib.Nifti1Image(posterior_lw,template.affine)
    nib.save(posteriornifti_lw,out+'layered_sbm_l'+str(l)+'_posterior_mean_lesionweight.nii.gz')

    sdnifti_lw = nib.Nifti1Image(sd_lw,template.affine)
    nib.save(sdnifti_lw,out+'layered_sbm_l'+str(l)+'_posterior_sd_lesionweight.nii.gz')
    
    impath = out+'layered_sbm_l'+str(l)+'_posterior_mean.nii.gz'
    fig=plotting.plot_glass_brain(impath, colorbar=True,
                          plot_abs=False,
                         threshold=0)
    
    impath = out+'layered_sbm_l'+str(l)+'_posterior_mean_lesionweight.nii.gz'
    fig=plotting.plot_glass_brain(impath, colorbar=True,
                          plot_abs=False,
                         threshold=0)

In [ ]:
parcels.dropna(inplace=True)
parcels.reset_index(drop=True,inplace=True)
parcels

In [ ]:
plt.figure(figsize=(5, 5))
sns.histplot(parcels['Target Error Posterior Mean'],bins=50)
plt.xlabel(task+' Posterior Mean')
plt.savefig(out+'block_posterior_means_target_error.png', dpi=600,bbox_inches='tight')

In [ ]:
plt.figure(figsize=(5, 5))
sns.histplot(parcels['Lesion Weight Posterior Mean'],bins=50)
plt.savefig(out+'block_posterior_means_target_error.png', dpi=600,bbox_inches='tight')

In [ ]:
temp=parcels.copy()
temp['CI Direction']=temp['Significant']
temp

for i, row in temp.iterrows():
    print(i)
    if temp.iloc[i,2] > temp.iloc[i,5]:
        print("target_error")
        temp.iloc[i,9] = 'Target Error Dominant Block'
    if temp.iloc[i,2] < temp.iloc[i,5]:
        print("LESION")
        temp.iloc[i,9] = 'Lesion Dominant Block'
        
    if temp.iloc[i,8] =='NSD':
        temp.iloc[i,9] = 'NSD'
        
# temp = temp[temp['Level']>0]
temp = temp[temp['Level']<6]
temp.reset_index(drop=True,inplace=True)
temp

In [ ]:
temp.loc[temp['CI Direction'] == '*', 'CI Direction']='NSD'

In [ ]:
temp[temp['CI Direction'] == '*']

In [ ]:
temp['CI Direction'].value_counts()

In [ ]:
plt.figure(figsize=(5, 5))
sns.scatterplot(data=temp,x='Target Error Posterior Mean',y='Lesion Weight Posterior Mean',hue='CI Direction',
               palette=["r", "b", "k"]).set(xlim=(None,0.75),ylim=(None,0.75))
plt.xlabel(task+' Posterior Mean')
plt.savefig(out+'block_posterior_means_scatter.png', dpi=600,bbox_inches='tight')

In [ ]:
print(parcels.shape)
parcels.head()
parcels.to_csv(out+'parcels_posteriors.csv')

In [ ]:
print(temp.shape)
temp.head()
temp.to_csv(out+'parcels_posteriors_withsiglabels.csv')

In [ ]:
parcel_looper = temp[temp['Level']==0].copy()
parcel_looper['Difference']=parcel_looper['Target Error Posterior Mean']-parcel_looper['Lesion Weight Posterior Mean']
parcel_looper[parcel_looper['Significant']!='NSD']

In [ ]:
plt.figure(figsize=(5, 5))
sns.histplot(parcel_looper['Difference'],bins=50)
plt.savefig(out+'block_posterior_means_difference.png', dpi=600,bbox_inches='tight')

In [ ]:
l=3

for l in range(6):
    print(l)
    parcel_looper = temp.copy()
    parcel_looper[parcel_looper['Level']==l]
    parcel_looper['Difference']=parcel_looper['Target Error Posterior Mean']-parcel_looper['Lesion Weight Posterior Mean']
    l_path = out+'layered_sbm_l'+str(l)+'.nii.gz'
    l_path = nib.load(l_path)
    l_path.affine
    limage= np.asanyarray(l_path.dataobj)
    # workthese = parcel_looper[parcel_looper['Significant']!='NSD'].copy()
    # workthese.reset_index(drop=True,inplace=True)
    # workthese

    parcel_looper.head()

    for i, row in parcel_looper.iterrows():
        block=parcel_looper.iloc[i,1]
        diff = parcel_looper.iloc[i,10]

        if parcel_looper.iloc[i,8]=='*':
            limage[limage==block]=diff
        if parcel_looper.iloc[i,8]=='NSD':
            limage[limage==block]=0

    limage[limage>1.5]=0

    diffnifti = nib.Nifti1Image(limage,template.affine)
    nib.save(diffnifti,out+'layered_sbm_l'+str(l)+'_difference_nifti.nii.gz')

In [ ]:
parcel_looper['CI Direction'].value_counts()

In [ ]:
parcel_looper[parcel_looper['CI Direction']=='target_error/NART Dominant Block']

In [ ]:
parcel_looper = temp[temp['Level']==0].copy()
parcel_looper['Difference']=parcel_looper['Target Error Posterior Mean']-parcel_looper['Lesion Weight Posterior Mean']
parcel_looper[parcel_looper['Significant']!='NSD']
parcel_looper

sns.color_palette("icefire", as_cmap=True)

plt.figure(figsize=(5, 5))
sns.scatterplot(data=parcel_looper,x='Target Error Posterior Mean',y='Lesion Weight Posterior Mean',hue='Difference',
               palette="icefire").set(xlim=(None,0.75),ylim=(None,0.75))
plt.xlabel(task+' Posterior Mean')
plt.savefig(out+'block_posterior_means_scatter_hue.png', dpi=600,bbox_inches='tight')

In [ ]:
parcel_looper = temp[temp['Level']==0].copy()
parcel_looper['Difference']=parcel_looper['Target Error Posterior Mean']-parcel_looper['Lesion Weight Posterior Mean']
parcel_looper[parcel_looper['Significant']!='NSD']
parcel_looper['Absolute Difference']=abs(parcel_looper['Difference'])

sns.color_palette("icefire", as_cmap=True)

plt.figure(figsize=(5, 5))
sns.scatterplot(data=parcel_looper,x='Target Error Posterior Mean',y='Lesion Weight Posterior Mean',hue='CI Direction',
               size='Absolute Difference').set(xlim=(None,0.75),ylim=(None,0.75))
plt.xlabel(task+' Posterior Mean')
plt.savefig(out+'block_posterior_means_scatter.png', dpi=600,bbox_inches='tight')

In [ ]:
plt.figure(figsize=(5, 5))
sns.scatterplot(data=parcel_looper,x='Target Error Posterior Mean',y='Lesion Weight Posterior Mean',hue='Difference',
               size='Absolute Difference',palette="icefire",
               legend = False).set(xlim=(None,0.4),ylim=(None,0.7))
plt.xlabel(task+' Posterior Mean')
plt.savefig(out+'block_posterior_means_scatter_hue.png', dpi=600,bbox_inches='tight')

In [ ]:
state.g.set_edge_filter(None)
thresh = np.percentile(state.g.ep.target_error.get_array(), 75)

target_error_edge_filter = g.new_edge_property("bool") 

for e in g.edges(): 
        if g.ep.target_error[e] >=thresh:
            target_error_edge_filter[e] = True 
        else: 
            target_error_edge_filter[e] = False 
# g.edge_properties["target_error_thresh_filter"]=edge_filter

In [ ]:
state.g.set_edge_filter(None)
thresh = np.percentile(state.g.ep.lw.get_array(), 75)

lw_edge_filter = g.new_edge_property("bool") 

for e in g.edges(): 
        if g.ep.lw[e] >=thresh:
            lw_edge_filter[e] = True 
        else: 
            lw_edge_filter[e] = False 
# g.edge_properties["target_error_thresh_filter"]=edge_filter

In [ ]:
state.g.set_edge_filter(None)
print(state.g)
state.g.set_edge_filter(target_error_edge_filter)
print(state.g)

In [ ]:
block1 = state.project_partition(0,0)
bg, bb, vcount, ecount, avp, aep = \
gt.condensation_graph(state.g,block1 ,self_loops=True)
vcount.get_array()
state.g.set_edge_filter(None)
state.g.set_edge_filter(state.g.ep.target_errorfilter)
print(state.g)

state.draw(edge_color=gt.prop_to_size(state.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
#            eorder=g.ep.target_error, edge_pen_width=gt.prop_to_size(g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
#            vertex_size=gt.prop_to_size(state.g.vp.target_error_x_eigen,1,200,power=1),
#            vertex_fill_color=gt.prop_to_size(state.g.vp.target_error_x_eigen,power=1),
#             vorder=state.g.vp.target_error_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_target_error.png',output_size=(4000,4000))

state.draw(edge_color=gt.prop_to_size(state.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
#            eorder=g.ep.target_error, edge_pen_width=gt.prop_to_size(g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
#            vertex_size=gt.prop_to_size(state.g.vp.target_error_x_eigen,1,200,power=1),
#            vertex_fill_color=gt.prop_to_size(state.g.vp.target_error_x_eigen,power=1),
#             vorder=state.g.vp.target_error_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_target_error.svg')

In [ ]:
block1 = state.project_partition(0,0)
bg, bb, vcount, ecount, avp, aep = \
gt.condensation_graph(state.g,block1 ,self_loops=True)
vcount.get_array()
state.g.set_edge_filter(None)
state.g.set_edge_filter(state.g.ep.lesionfilter)
print(state.g)

state.draw(edge_color=gt.prop_to_size(state.g.ep.lw, power=3, log=True), ecmap=(matplotlib.cm.inferno, .9),
           eorder=state.g.ep.lw, edge_pen_width=gt.prop_to_size(state.g.ep.lw, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
#            vertex_size=gt.prop_to_size(state.g.vp.lw_x_eigen,1,200,power=1),
#            vertex_fill_color=gt.prop_to_size(state.g.vp.lw_x_eigen,power=1),
#             vorder=state.g.vp.lw_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_lw.png',output_size=(4000,4000))

state.draw(edge_color=gt.prop_to_size(state.g.ep.lw, power=3, log=True), ecmap=(matplotlib.cm.inferno, .9),
           eorder=state.g.ep.lw, edge_pen_width=gt.prop_to_size(state.g.ep.lw, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
#            vertex_size=gt.prop_to_size(state.g.vp.lw_x_eigen,1,200,power=1),
#            vertex_fill_color=gt.prop_to_size(state.g.vp.lw_x_eigen,power=1),
#             vorder=state.g.vp.lw_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_lw.svg')

In [ ]:
state.draw()

In [ ]:
state.g.set_edge_filter(None)
state.g.set_vertex_filter(None)

t = gt.get_hierarchy_tree(state)[0]
# tpos = pos = gt.radial_tree_layout(t, t.vertex(t.num_vertices() - 1), weighted=True)
tpos = pos = gt.radial_tree_layout(t, t.vertex(0), weighted=True)
cts = gt.get_hierarchy_control_points(g, t, tpos)
pos = g.own_property(tpos)
b = state.levels[0].b
shape = b.copy()
shape.a %= len(np.unique(b.get_array()))

In [ ]:
gt.draw_hierarchy(state,pos=pos)

In [ ]:
# block1 = state.project_partition(0,0)
# bg, bb, vcount, ecount, avp, aep = \
# gt.condensation_graph(state.g,block1 ,self_loops=True)
# vcount.get_array()
state.g.set_edge_filter(None)
state.g.set_edge_filter(state.g.ep.target_errorfilter)
print(state.g)

state.draw(pos=pos,edge_color=gt.prop_to_size(state.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
           eorder=state.g.ep.target_error, edge_pen_width=gt.prop_to_size(state.g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.target_error_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.target_error_x_eigen,power=1),
            vorder=state.g.vp.target_error_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_target_error.png',output_size=(4000,4000))

state.draw(pos=pos,edge_color=gt.prop_to_size(state.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
           eorder=state.g.ep.target_error, edge_pen_width=gt.prop_to_size(state.g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.target_error_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.target_error_x_eigen,power=1),
            vorder=state.g.vp.target_error_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_target_error.svg')

In [ ]:
state_nullelayer.g.set_edge_filter(None)
state_nullelayer.g.set_edge_filter(state_nullelayer.g.ep.target_errorfilter)
print(state_nullelayer.g)

state_nullelayer.draw(edge_color=gt.prop_to_size(state_nullelayer.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
           eorder=state_nullelayer.g.ep.target_error, edge_pen_width=gt.prop_to_size(state_nullelayer.g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
          output=out+'null_layered_sbm_target_error.png',output_size=(4000,4000))

state_nullelayer.draw(edge_color=gt.prop_to_size(state_nullelayer.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
           eorder=state_nullelayer.g.ep.target_error, edge_pen_width=gt.prop_to_size(state_nullelayer.g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
          output=out+'null_layered_sbm_target_error.svg')

In [ ]:
# block1 = state.project_partition(0,0)
# bg, bb, vcount, ecount, avp, aep = \
# gt.condensation_graph(state.g,block1 ,self_loops=True)
# vcount.get_array()
state.g.set_edge_filter(None)
state.g.set_edge_filter(state.g.ep.target_errorfilter)
print(state.g)

state.draw(
    pos=pos,
           edge_color=gt.prop_to_size(state.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
           eorder=state.g.ep.target_error, edge_pen_width=gt.prop_to_size(state.g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.target_error_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.target_error_x_eigen,power=1),
            vorder=state.g.vp.target_error_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
#     hide=3,
          output=out+'layered_sbm_target_error_nohedge.png',output_size=(4000,4000))

state.draw(
    pos=pos,
           edge_color=gt.prop_to_size(state.g.ep.target_error, power=1, log=True), ecmap=(matplotlib.cm.inferno, .6),
           eorder=state.g.ep.target_error, edge_pen_width=gt.prop_to_size(state.g.ep.target_error, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.target_error_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.target_error_x_eigen,power=1),
            vorder=state.g.vp.target_error_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
#     hide=3,
          output=out+'layered_sbm_target_error_nohedge.svg')

In [ ]:
# block1 = state.project_partition(0,0)
# bg, bb, vcount, ecount, avp, aep = \
# gt.condensation_graph(state.g,block1 ,self_loops=True)
# vcount.get_array()
state.g.set_edge_filter(None)
state.g.set_edge_filter(state.g.ep.lesionfilter)
print(state.g)

state.draw(pos=pos,edge_color=gt.prop_to_size(state.g.ep.lw, power=3, log=True), ecmap=(matplotlib.cm.inferno, .9),
           eorder=state.g.ep.lw, edge_pen_width=gt.prop_to_size(state.g.ep.lw, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.lw_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.lw_x_eigen,power=1),
            vorder=state.g.vp.lw_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_lw.png',output_size=(4000,4000))

state.draw(pos=pos,edge_color=gt.prop_to_size(state.g.ep.lw, power=3, log=True), ecmap=(matplotlib.cm.inferno, .9),
           eorder=state.g.ep.lw, edge_pen_width=gt.prop_to_size(state.g.ep.lw, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.lw_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.lw_x_eigen,power=1),
            vorder=state.g.vp.lw_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
          output=out+'layered_sbm_lw.svg')

In [ ]:
state.g.set_edge_filter(None)
state.g.set_edge_filter(state.g.ep.lesionfilter)
print(state.g)

state.draw(pos=pos,edge_color=gt.prop_to_size(state.g.ep.lw, power=5, log=True), ecmap=(matplotlib.cm.inferno, .9),
           eorder=state.g.ep.lw, edge_pen_width=gt.prop_to_size(state.g.ep.lw, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.lw_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.lw_x_eigen,power=1),
            vorder=state.g.vp.lw_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
#            hide=20,
          output=out+'layered_sbm_lw_nohedge.png',output_size=(4000,4000))

state.draw(pos=pos,edge_color=gt.prop_to_size(state.g.ep.lw, power=5, log=True), ecmap=(matplotlib.cm.inferno, .9),
           eorder=state.g.ep.lw, edge_pen_width=gt.prop_to_size(state.g.ep.lw, 1, 4, power=1, log=True),
           edge_gradient=[],
          hvertex_size=15,hedge_pen_width=8,
           vertex_size=gt.prop_to_size(state.g.vp.lw_x_eigen,1,200,power=1),
           vertex_fill_color=gt.prop_to_size(state.g.vp.lw_x_eigen,power=1),
            vorder=state.g.vp.lw_x_eigen,
           vcmap=(matplotlib.cm.inferno, .6),
#            hide=20,
          output=out+'layered_sbm_lw_nohedge.svg')